In [6]:
%gui tk
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd

result_label = None

def open_file_dialog(method):
    file_path = filedialog.askopenfilename(filetypes=[("Fichiers Excel", "*.xlsx")])
    if file_path:
        messagebox.showinfo("Fichier sélectionné", "Vous avez sélectionné le fichier : " + file_path)
        # Traitement du fichier Excel
        data = pd.read_excel(file_path)
        print(data)
        data_dict=data.to_dict(orient='list')
        demand = data_dict['Unnamed: 1'][0]
        lead_time = data_dict['Unnamed: 1'][1]
        satisfaction = data_dict['Unnamed: 1'][2]

        if method == "Wilson en avenir certain sans contrainte":
            EOQ = ((2 * demand * lead_time) / satisfaction) ** 0.5
            result_label.config(text="Cout Moyen Total : " + str(EOQ))
        elif method == "Wilson en avenir certain avec contrainte":
            constraint = data_dict['Unnamed: 1'][3]
            EOQ = ((2 * demand * (lead_time + constraint)) / satisfaction) ** 0.5
            result_label.config(text="Cout moyen total : " + str(EOQ))
        elif method == "Wilson en avenir incertain":
            safety_factor = data_dict['Unnamed: 1'][3]
            EOQ = ((2 * demand * lead_time) / satisfaction) ** 0.5
            safety_stock = safety_factor * (demand * lead_time) ** 0.5
            result_label.config(text="Cout moyen total: " + str(EOQ) + "\nStock de sécurité : " + str(safety_stock))
        elif method == "Wilson avec coût de pénurie":
            shortage_cost = data_dict['Unnamed: 1'][3]
            EOQ = ((2 * demand * lead_time * shortage_cost) / satisfaction) ** 0.5
            result_label.config(text="Quantité économique de commande : " + str(EOQ))

def open_method_interface(method):
    method_window = tk.Toplevel(window)
    method_window.title(method)
    method_label = tk.Label(method_window, text="Interface pour la méthode de " + method)
    method_label.pack()
    global result_label
    result_label = tk.Label(method_window, text="")
    result_label.pack()

    open_data_interface(method)

def open_data_interface(method):
    data_window = tk.Toplevel(window)
    data_window.title("Choix des données")
    manual_button = tk.Button(data_window, text="Saisie manuelle des données", command=lambda: open_manual_entry(method))
    manual_button.pack()
    import_button = tk.Button(data_window, text="Importer un fichier Excel", command=lambda: open_file_dialog(method))
    import_button.pack()

def open_manual_entry(method):
    def validate_data():
        # Récupérer les valeurs saisies
        demand = int(entry1.get())
        lead_time = int(entry2.get())
        satisfaction = int(entry3.get())
        safety_factor = float(entry4.get())

        if method == "Wilson en avenir certain sans contrainte":
            EOQ = ((2 * demand * lead_time) / satisfaction) ** 0.5
            result_label.config(text="Cout Moyen Total : " + str(EOQ))
        elif method == "Wilson en avenir certain avec contrainte":
            constraint = int(entry5.get())
            EOQ = ((2 * demand * (lead_time + constraint)) / satisfaction) ** 0.5
            result_label.config(text="Cout moyen total : " + str(EOQ))
        elif method == "Wilson en avenir incertain":
            EOQ = ((2 * demand * lead_time) / satisfaction) ** 0.5
            safety_stock = safety_factor * (demand * lead_time) ** 0.5
            result_label.config(text="Cout moyen total: " + str(EOQ) + "\nStock de sécurité : " + str(safety_stock))
        elif method == "Wilson avec coût de pénurie":
            shortage_cost = float(entry5.get())
            EOQ = ((2 * demand * lead_time * shortage_cost) / satisfaction) ** 0.5
            result_label.config(text="Cout Moyen Total : " + str(EOQ))

    manual_window = tk.Toplevel(window)
    manual_window.title("Saisie manuelle des données")

    # Création des champs de saisie des données
    label1 = tk.Label(manual_window, text="Demande permanente:")
    label1.pack()
    entry1 = tk.Entry(manual_window)
    entry1.pack()

    label2 = tk.Label(manual_window, text="Délai de livraison:")
    label2.pack()
    entry2 = tk.Entry(manual_window)
    entry2.pack()

    label3 = tk.Label(manual_window, text="Satisfaction de la commande (%):")
    label3.pack()
    entry3 = tk.Entry(manual_window)
    entry3.pack()

    label4 = tk.Label(manual_window, text="Facteur de sécurité:")
    label4.pack()
    entry4 = tk.Entry(manual_window)
    entry4.pack()

    if method == "Wilson en avenir certain avec contrainte":
        label5 = tk.Label(manual_window, text="Contrainte")
        label5.pack()
        entry5 = tk.Entry(manual_window)
        entry5.pack()
    elif method == "Wilson avec coût de pénurie":
        label5 = tk.Label(manual_window, text="Coût de pénurie:")
        label5.pack()
        entry5 = tk.Entry(manual_window)
        entry5.pack()

    # Bouton de validation
    validate_button = tk.Button(manual_window, text="Valider", command=validate_data)
    validate_button.pack()

# Création de la fenêtre principale
window = tk.Tk()
window.title("Méthodes de Wilson")

# Création des boutons pour les méthodes de Wilson
button_wilson_sans_contrainte = tk.Button(window, text="Wilson en avenir certain sans contrainte", command=lambda: open_method_interface("Wilson en avenir certain sans contrainte"))
button_wilson_sans_contrainte.pack()

button_wilson_avec_contrainte = tk.Button(window, text="Wilson en avenir certain avec contrainte", command=lambda: open_method_interface("Wilson en avenir certain avec contrainte"))
button_wilson_avec_contrainte.pack()

button_wilson_avenir_incertain = tk.Button(window, text="Wilson en avenir incertain", command=lambda: open_method_interface("Wilson en avenir incertain"))
button_wilson_avenir_incertain.pack()

button_wilson_cout_penurie = tk.Button(window, text="Wilson avec coût de pénurie", command=lambda: open_method_interface("Wilson avec coût de pénurie"))
button_wilson_cout_penurie.pack()

# Lancement de la boucle principale de l'interface graphique
window.mainloop()